In [ ]:
import sys, os, time

In [ ]:
sys.path.insert(0,"../../python/")
import analyzer_utils as au

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from bokeh.io import output_notebook
output_notebook()

# Get collections

In [ ]:
raw_coll, db = au.get_coll("raw")
raw_begin_coll, _ = au.get_coll("raw_begining")

#### Number of documents in the collections

In [ ]:
raw_size = db.command("collstats", "raw")["count"]
raw_begin_size = db.command("collstats", "raw_begining")["count"]

In [ ]:
print("Number of docs in 'raw': %i" % raw_size)
print("Number of docs in 'raw_begining': %i" % raw_begin_size)

# Sample event

In [ ]:
doc = raw_coll.find_one()

In [ ]:
doc["created_at"]

from search_utils import tweet_time_2_epoch
tweet_time_2_epoch('Thu Jun 01 15:42:59 +0000 2017')

# Upper and lower bound of the available dataset

   * Qualifying days: May 22 - May 26 **First two days are missing!!! :(**
   * Normal contest: May 28 - June 11 
   * **I am still crawling events to see the decay in activity**

In [ ]:
au.find_some_docs(raw_begin_coll,sort_params=[("id",1)],limit=3)

In [ ]:
au.find_some_docs(raw_begin_coll,limit=3)

In [ ]:
au.find_some_docs(raw_coll,sort_params=[("id",1)],limit=3)

In [ ]:
au.find_some_docs(raw_coll,limit=3)

# Extract tweet text

In [ ]:
tweet_info_raw = au.get_tweets(raw_coll, without_urls=True)

In [ ]:
tweet_info_df_raw = pd.DataFrame(tweet_info_raw, columns=["tweet_id","time","source_id","source_name","language","text"])

In [ ]:
tweet_info_begin = au.get_tweets(raw_begin_coll, without_urls=True)

In [ ]:
tweet_info_df_begin = pd.DataFrame(tweet_info_begin, columns=["tweet_id","time","source_id","source_name","language","text"])

In [ ]:
tweet_info_df = pd.concat([tweet_info_df_raw, tweet_info_df_begin])

In [ ]:
tweet_info_df = tweet_info_df.sort_values("time")

tmp.ix[18362]["text"].replace("\r"," ")

tmp_eng = tweet_info_df[tweet_info_df["language"] == "en"]
len(tmp_eng)
#tmp = tmp_eng[tmp_eng["text"].apply(lambda x: "Man... " in x)]
tmp = tmp_eng[tmp_eng["text"].apply(lambda x: "Struggling to take it in " in x)]
len(tmp)
for idx, row in tmp.iterrows():
    print(idx, row, row["text"])#, row["text"].replace("\n"," "))

In [ ]:
tweet_info_df["source_name"] = tweet_info_df["source_name"].apply(lambda x: x.replace("|",""))
tweet_info_df["text"] = tweet_info_df["text"].apply(lambda x: x.replace("|",""))
tweet_info_df["text"] = tweet_info_df["text"].apply(lambda x: x.replace("\n"," "))
tweet_info_df["text"] = tweet_info_df["text"].apply(lambda x: x.replace("\r"," "))

In [ ]:
tweet_info_df.to_csv("/mnt/idms/fberes/network/roland_garros/data/rg17_tweets.csv",index=False,sep="|")

In [ ]:
tweet_info_df_en = tweet_info_df[tweet_info_df["language"] == "en"]
tweet_info_df_en.to_csv("/mnt/idms/fberes/network/roland_garros/data/rg17_tweets_eng.csv",index=False,sep="|")

# Extract mention networks

## a.) Extract mentions from multiple collections

In [ ]:
mentions_begin, user_names_begin, user_screen_names_begin, num_tweets_begin, num_retweets_begin = au.get_mentions(raw_begin_coll)

In [ ]:
mentions_df_begin = pd.DataFrame(mentions_begin,columns=["epoch","src","trg","lang","text"])
print(len(mentions_df_begin))

In [ ]:
mentions_df_begin.head()

In [ ]:
mentions_raw, user_names_raw, user_screen_names_raw, num_tweets_raw, num_retweets_raw = au.get_mentions(raw_coll)

In [ ]:
mentions_df_raw = pd.DataFrame(mentions_raw,columns=["epoch","src","trg","lang","text"])
print(len(mentions_df_raw))

## b.) Rolland Garros mention network (concatenated)
   * first part: May 23 16:51 -> May 31
   * second part: June 01 -> WIP

In [ ]:
mentions_df = pd.concat([mentions_df_begin, mentions_df_raw])
mentions_df = mentions_df.reset_index(drop=True)
mentions_df = mentions_df.sort_values("epoch")
user_names_begin.update(user_names_raw)
user_screen_names_begin.update(user_screen_names_raw)
user_names = user_names_begin
user_screen_names = user_screen_names_begin
num_tweets = num_tweets_begin + num_tweets_raw
num_retweets = num_retweets_begin + num_retweets_raw

In [ ]:
mentions_df.head()

In [ ]:
mentions_df["src_str"] =  mentions_df["src"].apply(lambda x: user_names[x])
mentions_df["trg_str"] =  mentions_df["trg"].apply(lambda x: user_names[x])
mentions_df["src_screen_str"] =  mentions_df["src"].apply(lambda x: user_screen_names[x])
mentions_df["trg_screen_str"] =  mentions_df["trg"].apply(lambda x: user_screen_names[x])

### 1. Number of mentions

In [ ]:
print("Number of tweets: %i" % num_tweets)
print("Number of retweets: %i" % num_retweets)
print("Number of mentions extracted from tweets: %i" % len(mentions_df))

### Export mentions to .csv

## Delete data after 2017-06-15 (I collected data until this point)

In [ ]:
mentions_df = mentions_df[mentions_df["epoch"] < 1497571200] # 2017-06-16 dawn UTC time

#### with recoded ID-s

In [ ]:
#filter for dates bigger than 2017 May 23 23:59 CET
#file_path = "/mnt/idms/fberes/network/roland_garros/data/rg17_mentions.csv"
file_path = "/mnt/idms/temporalNodeRanking/data/filtered_timeline_data/tsv/rg17/rg17_mentions.csv"
au.recode_and_export_mentions(file_path,mentions_df,user_names,epoch_lower_bound=1495576800)

#### with original ID-s

In [ ]:
cols_to_export = ["epoch","src","trg","src_str","src_screen_str","trg_str","trg_screen_str"]
mentions_df.to_csv("/mnt/idms/fberes/network/roland_garros/data/rg17_mentions_with_names.csv",columns=cols_to_export,sep="|",index=False)

In [ ]:
cols_to_export = ["epoch","src","trg","src_str","src_screen_str","trg_str","trg_screen_str","lang","text"]
mentions_df.to_csv("/mnt/idms/fberes/network/roland_garros/data/rg17_mentions_with_names_and_text.csv",columns=cols_to_export,sep="|",index=False)

### Read from .csv

In [ ]:
mentions_df = pd.read_csv("/mnt/idms/fberes/network/roland_garros/data/rg17_mentions_with_names.csv",sep="|")
mentions_df.head()

In [ ]:
src_map = dict(zip(mentions_df["src"],mentions_df["src_str"]))
trg_map = dict(zip(mentions_df["trg"],mentions_df["trg_str"]))
src_map.update(trg_map)
user_names = src_map

### 2. Popular source nodes

In [ ]:
au.show_frequent_items(mentions_df,user_names,"src",k=10)

### 3. Popular target nodes

In [ ]:
au.show_frequent_items(mentions_df,user_names,"trg",k=10)

### 4. Event distribution in time

In [ ]:
import time

In [ ]:
mentions_df["date"] = mentions_df["epoch"].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(x)))
mentions_df["time"] = mentions_df["epoch"].apply(lambda x: time.strftime('%H:%M:%S', time.localtime(x)))

In [ ]:
mentions_df.head()

In [ ]:
plt.figure(figsize=(15,5))
plt.title("Number of mentions in time")
mentions_df["epoch"].hist(bins=500)
plt.show()

In [ ]:
num_of_mentions_by_day = mentions_df["date"].value_counts().sort_index()

In [ ]:
plt.figure(figsize=(15,5))
plt.title("Number of mentions per day")
plt.plot(num_of_mentions_by_day.values)
plt.xticks(range(len(num_of_mentions_by_day)),num_of_mentions_by_day.index,rotation='vertical')
plt.show()

### 5. Graph informations

In [ ]:
date_splits = sorted(list(mentions_df["date"].unique()))
#date_splits

In [ ]:
graph_stats = [] 
for split in date_splits[1:]:
    partial_df = mentions_df[mentions_df["date"] < split]
    graph_stats += [au.get_graph_stats(partial_df)]
graph_stats_df = pd.DataFrame(graph_stats, columns=["nodes","edges","weak_components","strong_components"])

In [ ]:
graph_stats_df.head(3)

In [ ]:
plt.subplots(figsize=(16,5))

plt.subplot(1,2,1)
plt.title('Aggregated mention graph size')
plt.plot(graph_stats_df["nodes"],label="nodes")
plt.plot(graph_stats_df["edges"],label="edges")
plt.xticks(range(len(graph_stats_df)),date_splits[:-1],rotation='vertical')
plt.legend()

plt.subplot(1,2,2)
plt.title('Number of weak components in the aggregated mention graph')
plt.plot(graph_stats_df["weak_components"],label="number of ",c="r")
plt.xticks(range(len(graph_stats_df)),date_splits[:-1],rotation='vertical')
plt.show()

### 6. Changes in popular items

In [ ]:
day_list = []
day_list += ['2017-05-%i' % i for i in range(23,32)]
day_list += ['2017-06-0%i' % i for i in range(1,10)]
#day_list += ['2017-06-1%i' % i for i in range(0,3)]
day_list

In [ ]:
k = 20
popular_trgs = []
for i in range(len(day_list)):
    daily_df = mentions_df[mentions_df["date"]==day_list[i]]
    freq_trg = au.show_frequent_items(daily_df,user_names,"trg",k=k)
    popular_trgs += list(zip(np.ones(k)*i,freq_trg["name"],freq_trg["count"],freq_trg["count"]/len(daily_df)))
    #popular_trgs += list(zip(np.ones(k)*i,freq_trg["name"],np.sqrt(freq_trg["count"])))
    #popular_trgs += list(zip(np.ones(k)*i,freq_trg["name"],np.log(freq_trg["count"])))
popular_trg_df = pd.DataFrame(popular_trgs, columns=["day_idx","name","count","dominance"])

#### At least 3 times in top_20
   * 06-09:
      * Wawrinka defeated Murray
      * Nadal defeated Dominic Thiem

In [ ]:
au.plot_user_popularity(au.filter_for_support(popular_trg_df, min_times=3),day_list)

#### At least 8 times in top_20

   * Nadal and Wawrinka will play for 1st and 2nd prize
   * Wawrinka came from very low, he was not among top_20 in 10 days...
   * Djokovic was defeated on 06-07 by Dominic Thiem
   * 06-09:
      * Wawrinka defeated Murray
      * Nadal defeated Dominic Thiem

In [ ]:
very_pop_df = au.filter_for_support(popular_trg_df, min_times=8)
#au.plot_user_popularity(very_pop_df, day_list)
au.plot_user_dominance(very_pop_df)

#### At least 4 times in top_20 but less then 8 times

In [ ]:
very_pop_df = au.filter_for_support(popular_trg_df, max_times=7, min_times=4)
#plot_user_popularity(very_pop_df,day_list)
au.plot_user_dominance(very_pop_df)